### Подготовка данных

In [6]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_data.head(2)

,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"


In [18]:
melb_df = melb_data.copy()

#### Удаление лишних столбцов

* df.**drop()**, где:
    + labels — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;
    + axis — ось совершения операции, axis=0 — удаляются строки, axis=1 — удаляются столбцы;
    + inplace — если параметр выставлен на True, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает; если на False — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; по умолчанию параметр равен False.

In [19]:
# дублирующийся параметр с координатами, ненужный стобец с индексом
melb_df.drop(['index','Coordinates'], axis=1, inplace=True) 

#### Добавление новых признаков

In [20]:
# добавление признака "Средняя площадь комнаты"
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'].head(3))

0    25.20
1    15.80
2    18.75
Name: MeanRoomsSquare, dtype: float64

In [21]:
# добавление признака "Коэффициент соотношения площади здания и площади участка"
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'].head(3))

0   -0.231707
1   -0.327660
2    0.056338
Name: AreaRatio, dtype: float64

#### Преобразование дат

Составлющие datetime:
* date — дата;
* year, month, day — год, месяц, день;
* time — время;
* hour, minute, second — час, минута, секунда;
* dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
* day_name — название дня недели;
* dayofyear — порядковый день года;
* quarter — квартал (интервал в три месяца).

In [22]:
display(melb_df['Date'].head(1)) # до преобразования
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'].head(1)) # после преобразования

0    3/12/2016
Name: Date, dtype: object

0   2016-12-03
Name: Date, dtype: datetime64[ns]

In [23]:
years_sold = melb_df['Date'].dt.year
print('Min year sold:', years_sold.min())

Min year sold: 2016


In [24]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days.head(2)) # тип timedelta

0   337 days
1    34 days
Name: Date, dtype: timedelta64[ns]

In [34]:
# Создайте в таблице melb_df признак WeekdaySale (день недели). 
# #Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]

12822

In [39]:
ufo_data = pd.read_csv('data/ufo.csv', sep=',')

In [42]:
# В каком году отмечается наибольшее количество случаев наблюдения НЛО в США?
ufo_data['DateTime'] = pd.to_datetime(ufo_data['Time'], dayfirst=True)
ufo_data['DateTime'].dt.year.mode()

0    1999
Name: DateTime, dtype: int64

Признаки со слишком большим кол-вом значений, скорее всего, не имеют статистической значимости.
Такие признаки можно удалить из модели, либо получить из них значимую информацию.

In [59]:
# Выделим из адреса объекта тип улицы (улица, шоссе, авеню, бульвар)
def get_street_type(address):
    # Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
    address_list = address.split(' ')
    street_type = address_list[-1]
    # Делаем проверку на то, что полученный подтип является географической пометкой.
    # Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
        street_type = address_list[-2]
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
display(street_types.head(4))

0    St
1    St
2    St
3    La
Name: Address, dtype: object

In [60]:
# Проверим, сколько уникальных значений
print(street_types.nunique())

56


In [61]:
# Проверим, равномерноли они распределены
display(street_types.value_counts())

St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Corso     

In [63]:
# Получим 10 наиболее популрных значений, остальным присвоим значение other
popular_stypes =street_types.value_counts().nlargest(10).index
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')

In [64]:
# Удалим лишнюю колонку Address
melb_df = melb_df.drop('Address', axis=1)

In [95]:
# Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: 
# оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
# Сравните минимальные цены объектов недвижимости, проданных компанией 'Nelson', и компаниями, обозначенными как 'other'.
unique_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in unique_sellers else 'other')
melb_df[(melb_df['SellerG'] == 'Nelson') | (melb_df['SellerG'] == 'other')].groupby(['SellerG'])['Price'].min()


SellerG
Nelson    170000.0
other     131000.0
Name: Price, dtype: float64

#### Типы признаков

* **числовые** - отражают количественную меру и могут принимать значения из неограниченного диапазона:
    + **дискретные** - например, количество комнат, пациентов, дней, отток сотрудников;
    + **непрерывными** - например, масса, цена, площадь;
* **категориальные** - обозначают принадлежность объекта к какому-то классу/категории и имеют ограниченный набор значений:
    + **номинальные** - например, пол, национальность, район;
    + **порядковые** - например, уровень образования, уровень комфорта, стадия заболевания.    

In [96]:
# определим число уникальных категорий в каждом столбце нашей таблицы melb_df
unique_list = []
for col in melb_df.columns:
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts)

,Column_Name,Num_Unique,Type
0,Weekend,2,int64
1,Type,3,object
2,Method,5,object
3,WeekdaySale,5,int64
4,Regionname,8,object
5,Bathroom,9,int64
6,Rooms,9,int64
7,Car,11,int64
8,StreetType,11,object
9,Bedroom,12,int64


**Category** - тип данных для работы с категориальными признаками.

Тип данных внешне выглядит как строка, но внутренне представлен массивом целых чисел. 
Данные вместо строк хранятся в памяти как число, объём памяти, занимаемой таблицей при использовании типа category, резко уменьшается.

Тип данных расширяет возможности работы с категориальными признаками: мы можем легко преобразовывать категории, строить графики по таким данным. Также резко повышается производительность операций, совершаемых с такими столбцами.

In [110]:
# преобразуем к category те столбцы, в которых более 150 уникальных значений, кроме числовых
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: 
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info()) # размер 1.8 мб, до перобразования 2.6

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  object        
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  YearBuilt        13580 non-null  categ

None

#### Получение атрибутов categories

У типа данных category есть свой специальный аксесcор cat, который позволяет получать информацию о своих значениях и преобразовывать их.

In [111]:
melb_df['Regionname'].cat.categories # список категорий

Index(['Eastern Metropolitan', 'Eastern Victoria', 'Northern Metropolitan',
       'Northern Victoria', 'South-Eastern Metropolitan',
       'Southern Metropolitan', 'Western Metropolitan', 'Western Victoria'],
      dtype='object')

In [117]:
melb_df['Regionname'].head(2).cat.codes # как категории кодируются в памяти

0    2
1    2
dtype: int8

In [116]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({ # переименование категорий
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
melb_df['Type'].head(2)

0    house
1    house
Name: Type, dtype: category
Categories (3, object): ['house', 'townhouse', 'unit']

Тип данных category хранит только категории, которые были объявлены при его инициализации. При встрече с новой, неизвестной ранее категорией, этот тип превратит её в пустое значение.

Если набор категорий в столбце жёстко не зафиксирован и может обновляться в процессе работы, то тип category не является подходящим типом данных для этого столбца

In [118]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat') # добавление новой категории
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)

In [120]:
unique_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in unique_suburb else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
melb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  YearBuilt        13580 non-null  categ